In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/refs/heads/main/helper_functions.py

--2025-04-09 06:06:20--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-04-09 06:06:20 (49.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [ ]:
# Download data from Google Storage (already preformatted)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

unzip_data('101_food_classes_10_percent.zip')

train_dir = '101_food_classes_10_percent/train/'
test_dir = '101_food_classes_10_percent/test/'

--2025-04-09 06:06:36--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.207, 74.125.202.207, 74.125.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   126MB/s    in 18s     

2025-04-09 06:06:54 (86.9 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [ ]:
import tensorflow as tf

IMG_SIZE= (224,224)
batch_size=32

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                image_size=IMG_SIZE,
                                                                                label_mode='categorical',
                                                                                batch_size=batch_size)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                image_size=IMG_SIZE,
                                                                label_mode='categorical',
                                                                shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [ ]:
walk_through_dir('/content/101_food_classes_10_percent')

There are 2 directories and 0 images in '/content/101_food_classes_10_percent'.
There are 101 directories and 0 images in '/content/101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/chocolate_cake'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/pad_thai'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/spaghetti_carbonara'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/greek_salad'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/crab_cakes'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/mussels'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/tiramisu'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/beignets'.
There are 0 directories and 250 images in '/

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2)
],name='data_augmentation')

In [ ]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable=False

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
input_shape = (224,224,3)
inputs = tf.keras.layers.Input(shape=input_shape,name='input_layer')
x = data_augmentation(inputs)
x = base_model(x,training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
outputs = tf.keras.layers.Dense(len(train_data_all_10_percent.class_names),activation='softmax',name='output_layer')(x)

In [ ]:
model_0 = tf.keras.Model(inputs,outputs)

In [ ]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                metrics=['accuracy'])

In [ ]:
history_0 = model_0.fit(train_data_all_10_percent,
                        epochs=5,
                        steps_per_epoch=int(len(train_data_all_10_percent)),
                        validation_data=test_data,
                        validation_steps=int(0.25 * len(test_data)))

Epoch 1/5
  6/237 ━━━━━━━━━━━━━━━━━━━━ 7:42 2s/step - accuracy: 0.0128 - loss: 4.7106

In [ ]:
plot_loss_curves(history_0)

# Fine-Tuning

In [ ]:
model_0_best_model = model_0.layers[2]

In [ ]:
model_0_best_model.trainable = False

In [ ]:
for layer in (model_0_best_model.layers[-5:]):
  layer.trainable = True

In [ ]:
for layer_number, layer in enumerate(model_0_best_model.layers):
  print(layer_number, '|', layer.name, '|', layer.trainable)

In [ ]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                metrics=['accuracy'])

In [ ]:
checkpoint_path = '101_classes_10_percent_data_model_checkpoint.weights.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)

In [ ]:
fine_tune_epochs = 10

history_all_classes_10_percent_fine_tune = model_0.fit(train_data_all_10_percent,
                                                       epochs=fine_tune_epochs,
                                                       steps_per_epoch=int(len(train_data_all_10_percent)),
                                                       validation_steps=int(0.25 * len(test_data)),
                                                       initial_epoch=history_0.epoch[-1],
                                                       callbacks=[checkpoint_callback])

In [ ]:
for layer in model_0.layers:
  print(layer.name, layer.trainable)

In [ ]:
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number, layer.name, layer.trainable)

In [ ]:
results_all_classes_10_percent_fine_tune = model_0.evaluate(test_data)
results_all_classes_10_percent_fine_tune

In [ ]:
plot_loss_curves(history_all_classes_10_percent_fine_tune)

In [ ]:
def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two model history objects.
    """
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    print(len(acc))

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    print(len(total_acc))
    print(total_acc)

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
compare_historys(original_history=history_0,
                 new_history=history_all_classes_10_percent_fine_tune,
                 initial_epochs=5)

# Error Analysis

In [ ]:
pred_probs = model_0.predict(test_data,verbose=1)

In [ ]:
len(pred_probs)

In [ ]:
pred_probs.shape

In [ ]:
pred_probs[0]

In [ ]:
pred_classes = pred_probs.argmax(axis=1)
pred_classes[:10]

In [ ]:
y_labels = []
for image, labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())

In [ ]:
y_labels[:10]

In [ ]:
pred_classes[:10]

In [ ]:
from sklearn.metrics import accuracy_score

sklearn_accuracy = accuracy_score(y_labels,pred_classes)
sklearn_accuracy

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):

  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)


  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes),
         yticks=np.arange(n_classes),
         xticklabels=labels,
         yticklabels=labels)


  # ax.xaxis.set_label_position("bottom")
  # ax.xaxis.tick_bottom()


  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)


  threshold = (cm.max() + cm.min()) / 2.


  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

In [ ]:
class_names = test_data.class_names
class_names[:10]

In [ ]:
make_confusion_matrix(y_true_labels,
                      y_pred=pred_classes,
                      classes=class_names,
                      figsize=(100, 100),
                      text_size=20)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels,pred_classes))

In [ ]:
classification_report_dict = classification_report(y_labels,pred_classes,output_dict=True)
classificaion_report_dict

In [ ]:
class_f1_scores = {}

for k, v in classification_report_dict.items():
  if k == 'accuracy':
    break
  else:
    class_f1_scores[class_names[int(k)]] == v['f1-score']
class_f1_scores

In [ ]:
f1_scores = pd.DataFrame({'class_name': list(class_f1_scores.keys()),
                          'f1-score': list(class_f1_scores.values())}).sort_values('f1-score',ascending=False)
f1_scores.head()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(list(f1_scores["class_name"]))
ax.set_xlabel("f1-score")
ax.set_title("F1-Scores for 10 Different Classes")
ax.invert_yaxis();

def autolabel(rects):
  """
  Attach a text label above each bar displaying its height (it's value).
  """
  for rect in rects:
    width = rect.get_width()
    ax.text(1.03*width, rect.get_y() + rect.get_height()/1.2,
            f"{width:.2f}",
            ha='center', va='bottom')

autolabel(scores)

# As we can see apple pie is the worst category for predictions. In order to fix this issue we can obtain more data

In [ ]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  img = tf.io.read_file(filename)
  img = tf.io_decode_image(img)

  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    return img/255
  else:
    return img

In [ ]:
import os
import random

plt.figure(figsize=(17,10))
for i in range(3):
  # Choose a random image from a random class
  class_name = random.choice(class_names)
  filename = random.choice(os.listdir(test_dir + '/' + class_name))
  filepath = test_dir + class_name + '/' + filename

  # Load the image and make predictions
  img = load_and_prep_image(filepath, scale=False)
  pred_prob = model_0.predict(tf.expand_dims(img, axis=0))
  pred_class = class_names[pred_prob.argmax()]

  # Plot the image(s)
  plt.subplot(1, 3, i+1)
  plt.imshow(img/255.)
  if class_name == pred_class:
    title_color = 'g'
  else:
    title_color = 'r'
  plt.title(f'actual: {class_name}, pred: {pred_class}, prob: {pred_prob.max():.2f}', c=title_color)
  plt.axis(False);

 # Now let's look for biggest errors

In [ ]:
filepaths = []
for filepath in test_data.list_files("101_food_classes_10_percent/test/*/*.jpg",
                                     shuffle=False):
  filepaths.append(filepath.numpy())
filepaths[:10]

In [ ]:
pred_df = pd.DataFrame({'img_path': filepaths,
                        'y_true': y_labels,
                        'y_predict': pred_classes,
                        'pred_conf': pred_probs.max(axis=1),
                        'y_true_classname': [class_names[i] for i in y_labels],
                        'y_pred_classname': [class_names[i] for i in pred_classes]})
pred_df.head()

In [ ]:
pred_df['inequal'] = (pred_df['y_true_classname'] != pred_df['y_pred_classname'])

In [ ]:
inequal_table = pred_df[pred_df['inequal'] == True]
inequal_table.sort_values(bye='pred_conf',ascending=False,inplace=True)
inequal_table

In [ ]:
images_to_view = 9
start_index = 10 # change the start index to view more
plt.figure(figsize=(15,10))
for i, row in enumerate(inequal_table[start_index : start_indrx + images_to_view].iterplt.subplot()):
  plt.subplot(3, 3, i+1)
  img = load_and_prep_image(row[1], sclae=True)
  _, _, _, _, pred_prob, y_true, y_pred, _ = row # only interested in a few parameters for each row
  plt.imshow(img)
  plt.title(f'actua;: {y_true}, pred: {y_pred} \nprob: {pred_prob:.2f}')
  plt.axis(False)